In [1]:
# Import packages

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from glicko2 import Player


# Import csv file

fighter_df = pd.read_csv('C:/Users/12152/Box/Other/full_fight_df.csv')

In [3]:
from datetime import timedelta

# Set parameters
decay_rate = 5              # Elo points per 120 days
period_days = 120           # Length of decay period
baseline_elo = 1500         # Starting Elo

elo_ratings = {}            # Maps fighter to current Elo
last_fight_date = {}        # Maps fighter to last fight date
elo_history = []

for _, row in fighter_df.sort_values('date').iterrows():
    f1, f2 = row['fighter'], row['opponent']
    outcome = row['result']
    fight_date = pd.to_datetime(row['date'])

    # Initialize fighters
    for fighter in [f1, f2]:
        if fighter not in elo_ratings:
            elo_ratings[fighter] = baseline_elo
            last_fight_date[fighter] = fight_date

    # Apply decay for inactivity before this fight
    for fighter in [f1, f2]:
        days_inactive = (fight_date - last_fight_date[fighter]).days
        decay_periods = max(0, days_inactive // period_days)
        elo_ratings[fighter] -= decay_periods * decay_rate
        elo_ratings[fighter] = max(1300, elo_ratings[fighter])  # optional floor

    # Elo update logic (basic version)
    ra, rb = elo_ratings[f1], elo_ratings[f2]
    k = 32

    ea = 1 / (1 + 10 ** ((rb - ra) / 400))
    eb = 1 - ea

    if outcome == 'Win':
        sa, sb = 1.0, 0.0
    elif outcome == 'Loss':
        sa, sb = 0.0, 1.0
    else:
        sa = sb = 0.5

    elo_ratings[f1] += k * (sa - ea)
    elo_ratings[f2] += k * (sb - eb)

    # Update last fight dates
    last_fight_date[f1] = fight_date
    last_fight_date[f2] = fight_date

    # Store history
    elo_history.append({
        'date': fight_date,
        'fighter': f1,
        'opponent': f2,
        'elo': elo_ratings[f1],
        'result': outcome
    })


In [4]:
elo_df = pd.DataFrame(elo_history)

In [6]:
max_elo_df = (
    elo_df.groupby('fighter')['elo']
    .max()
    .sort_values(ascending=False)
    .rename('max_elo')
    .round(2)
    .reset_index()
)

print(max_elo_df.head(25))

                     fighter  max_elo
0          Fedor Emelianenko  1948.19
1             Anderson Silva  1931.45
2                  Jon Jones  1922.99
3          Georges St-Pierre  1921.19
4              Travis Fulton  1905.82
5             Daniel Cormier  1894.82
6               Kamaru Usman  1893.66
7            Islam Makhachev  1892.35
8             Donald Cerrone  1887.64
9   Antônio Rodrigo Nogueira  1884.25
10                 José Aldo  1875.78
11          Charles Oliveira  1867.39
12       Khabib Nurmagomedov  1867.35
13               Renan Barão  1866.07
14     Alexander Volkanovski  1864.36
15        Demetrious Johnson  1857.71
16               Matt Hughes  1855.49
17           Wanderlei Silva  1854.51
18                Ryan Bader  1852.21
19            Dustin Poirier  1846.95
20          Benson Henderson  1844.38
21             Tony Ferguson  1843.59
22              Jake Shields  1839.76
23              Max Holloway  1839.31
24          Igor Vovchanchyn  1835.23


In [2]:
import numpy as np
from datetime import timedelta

# Parameters
baseline_elo = 1500
decay_lambda = 0.00015  # Decay speed — tune this!
elo_ratings = {}
last_fight_date = {}
elo_history2 = []

for _, row in fighter_df.sort_values('date').iterrows():
    f1, f2 = row['fighter'], row['opponent']
    outcome = row['result']
    fight_date = pd.to_datetime(row['date'])

    for fighter in [f1, f2]:
        if fighter not in elo_ratings:
            elo_ratings[fighter] = baseline_elo
            last_fight_date[fighter] = fight_date

        # Apply exponential decay
        days_inactive = (fight_date - last_fight_date[fighter]).days
        if days_inactive > 0:
            decay_factor = np.exp(-decay_lambda * days_inactive)
            elo_ratings[fighter] = baseline_elo + (elo_ratings[fighter] - baseline_elo) * decay_factor

    # Elo update
    ra, rb = elo_ratings[f1], elo_ratings[f2]
    k = 32
    ea = 1 / (1 + 10 ** ((rb - ra) / 400))
    eb = 1 - ea

    if outcome == 'Win':
        sa, sb = 1.0, 0.0
    elif outcome == 'Loss':
        sa, sb = 0.0, 1.0
    else:
        sa = sb = 0.5

    elo_ratings[f1] += k * (sa - ea)
    elo_ratings[f2] += k * (sb - eb)

    last_fight_date[f1] = fight_date
    last_fight_date[f2] = fight_date

    elo_history2.append({
        'date': fight_date,
        'fighter': f1,
        'opponent': f2,
        'elo': elo_ratings[f1],
        'result': outcome
    })

elo_decay_df = pd.DataFrame(elo_history2)


In [3]:
max_elo_decay_df = (
    elo_decay_df.groupby('fighter')['elo']
    .max()
    .sort_values(ascending=False)
    .rename('max_elo')
    .round(2)
    .reset_index()
)

print(max_elo_decay_df.head(25))

                     fighter  max_elo
0          Fedor Emelianenko  1914.09
1             Anderson Silva  1907.83
2          Georges St-Pierre  1898.25
3                  Jon Jones  1896.82
4               Kamaru Usman  1891.20
5            Islam Makhachev  1888.31
6              Travis Fulton  1888.07
7             Daniel Cormier  1878.15
8   Antônio Rodrigo Nogueira  1874.66
9                  José Aldo  1874.33
10            Donald Cerrone  1871.81
11          Charles Oliveira  1866.00
12     Alexander Volkanovski  1864.03
13       Khabib Nurmagomedov  1859.09
14        Demetrious Johnson  1857.86
15               Renan Barão  1856.52
16                Ryan Bader  1848.61
17             Tony Ferguson  1847.63
18              Jake Shields  1846.72
19              Max Holloway  1844.58
20           Israel Adesanya  1843.60
21           Wanderlei Silva  1842.04
22            Belal Muhammad  1841.45
23            Dustin Poirier  1841.19
24          Benson Henderson  1839.19
